In [1]:
import pandas as pd 
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer  # Enable IterativeImputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import lightgbm as lgb
import optuna
from sklearn.ensemble import StackingRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [2]:
data = pd.read_csv('/kaggle/input/aoml-m-2-test-feb-2025/train.csv')
data = pd.get_dummies(data, columns=['day'], drop_first=True, dtype=int)

In [3]:
X = data.drop(['output_electricity_generation'],axis=1)
y = data[['output_electricity_generation']]

train_imputer = IterativeImputer(
    estimator=XGBRegressor(n_estimators=120, learning_rate=0.1, max_depth=10, random_state=42), 
    max_iter=12,
    random_state=42
)

X_imputed = train_imputer.fit_transform(X) 
X_imp_df = pd.DataFrame(X_imputed, columns=X.columns)

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [5]:
X_imp_df.drop(['uid'],axis=1,inplace=True)
X_imp_df.shape

(50400, 36)

In [8]:
poly_feat_creator = PolynomialFeatures(degree=2, include_bias=False)

poly_feat_train = poly_feat_creator.fit_transform(X_imp_df)
poly_feat_test = poly_feat_creator.transform(test_df_imp)

print("Poly train shape", poly_feat_train.shape)
print("Poly test shape", poly_feat_test.shape)

Poly train shape (50400, 702)
Poly test shape (21600, 702)


In [9]:
mmscaler = MinMaxScaler()
yscaler = MinMaxScaler()

X_scl = mmscaler.fit_transform(poly_feat_train)
y_scl = yscaler.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_scl, y_scl, test_size=0.1, random_state=42)

In [10]:
from sklearn.ensemble import RandomForestRegressor

# Define parameters for Random Forest
rf_params = {
    'n_estimators': 100,    # Equivalent to num_round in XGBoost
    'max_depth': 8,         # Similar to XGBoost's max_depth
    'min_samples_split': 5, # Equivalent to controlling overfitting
    'min_samples_leaf': 2,  # Controls leaf size, like regularization
    'max_features': 0.7,    # Similar to colsample_bytree
    'max_samples': 0.7,     # Similar to subsample
    'n_jobs': -1,           # Use all processors
    'random_state': 42
}

# Train the model
rf_model = RandomForestRegressor(**rf_params)
rf_model.fit(X_train, y_train)

# Predict on test set
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {np.sqrt(mse)}')

<ipython-input-10-aa060268c009>:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train, y_train)


RMSE: 0.054339087661661976


In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer
import numpy as np

# Define RMSE as the evaluation metric
rmse_scorer = make_scorer(mean_squared_error, squared=False)

# Define Random Forest parameters
rf_params = {
    'n_estimators': 100,
    'max_depth': 8,
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'max_features': 0.7,
    'max_samples': 0.7,
    'n_jobs': -1,
    'random_state': 42
}

# Initialize the RandomForestRegressor
rf_model = RandomForestRegressor(**rf_params)

# Perform 5-Fold Cross-Validation
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring=rmse_scorer)

# Print results
print(f"Cross-Validation RMSE Scores: {cv_scores}")
print(f"Mean RMSE: {np.mean(cv_scores):.4f}")
print(f"Standard Deviation of RMSE: {np.std(cv_scores):.4f}")


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed whe

Cross-Validation RMSE Scores: [0.0033037  0.00300496 0.00266167 0.00267846 0.00286759]
Mean RMSE: 0.0029
Standard Deviation of RMSE: 0.0002


In [13]:
rf_params

{'n_estimators': 100,
 'max_depth': 8,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 0.7,
 'max_samples': 0.7,
 'n_jobs': -1,
 'random_state': 42}

In [14]:
# Re-train the model on the full training data
from sklearn.model_selection import cross_val_predict
# Get cross-validation predictions
y_train_pred = cross_val_predict(rf_model, X_train, y_train, cv=5)
# Train the best model on the full training data
rf_model.fit(X_train, y_train)

# Predict on test set
y_pred = rf_model.predict(X_test)


# Predict on the test set
mse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {np.sqrt(mse)}')

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:1068: DataConversionWarning: A column-vector y was passed

RMSE: 0.054339087661662


In [18]:
best_n_estimators = 1999
def objective(trial):
    params = {
        'n_estimators': best_n_estimators,
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.2),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 5.0),
    }
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    model = xgb.train(params, dtrain, num_boost_round=params['n_estimators'])
    
    preds = model.predict(dtest)
    return mean_squared_error(y_test, preds, squared=False) # RMSE

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=15)

best_params = study.best_params

[I 2025-02-09 12:12:18,131] A new study created in memory with name: no-name-65833556-b752-4457-acf0-b9dc5b41b96c
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:12:18] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2025-02-09 12:12:31,893] Trial 0 finished with value: 0.0026745383435352464 and parameters: {'learning_rate': 0.14732908947642276, 'max_depth': 5, 'subsample': 0.8476089054623736, 'colsample_bytree': 0.7305993735376146, 'reg_alpha': 0.5812000930242557, 'reg_lambda': 3.4948048158713974}. Best is trial 0 with value: 0.0026745383435352464.
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:12:31] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2025-02-09 12:12:48,053] Trial 1 finished with value: 0.0025499964454778657 and parameters: {'learning_rate': 0.107801668139298

Trial 11 finished with value: 0.0013545071142224234 and parameters: {'learning_rate': 0.01861885913259459, 'max_depth': 10, 'subsample': 0.8358619178926193, 'colsample_bytree': 0.8403892834887154, 'reg_alpha': 0.002382874488102673, 'reg_lambda': 0.6066665118899315}.

Trial 11 finished with value: 0.0016639691136858914 and parameters: {'learning_rate': 0.08846867966096117, 'max_depth': 13, 'subsample': 0.6191093025892407, 'colsample_bytree': 0.6963141610127959, 'reg_alpha': 0.049171806546736925, 'reg_lambda': 2.9803541719046174}. Best is trial 11 with value: 0.0016639691136858914.

Trial 10 finished with value: 0.0015019409003177613 and parameters: {'learning_rate': 0.005874302462010629, 'max_depth': 14, 'subsample': 0.9671985420036247, 'colsample_bytree': 0.504163927700474, 'reg_alpha': 0.0026961225198311456, 'reg_lambda': 4.802038438411944}. Best is trial 10 with value: 0.0015019409003177613.

In [19]:
dtrain = xgb.DMatrix(X_train, label=y_train)

# Define parameters for XGBoost
params = {
    'objective': 'reg:squarederror',
    'n_estimators': 1999,
    'learning_rate': 0.08846867966096117,   # Smaller learning rate improves generalization
    'max_depth': 13,          # Higher depth captures more interactions
    'subsample':  0.6191093025892407,        # Prevents overfitting
    'colsample_bytree': 0.6963141610127959, # Feature selection
    'reg_alpha': 0.049171806546736925,        # L1 regularization
    'reg_lambda': 2.9803541719046174,       # L2 regularization
    'n_jobs': -1
}
num_round = 100

# Train the model

# bst = xgb.train(params, dtrain, num_round)
xgb_model = XGBRegressor(**best_params)
lgb_model = LGBMRegressor(n_estimators=500, learning_rate=0.05)
cat_model = CatBoostRegressor(n_estimators=500, learning_rate=0.05, verbose=0)

stack_model = StackingRegressor(estimators=[
    ('xgb', xgb_model),
], final_estimator=XGBRegressor(n_estimators=100))

stack_model.fit(X_train, y_train)

# dtest = xgb.DMatrix(X_test)
# predictions = bst.predict(dtest)
predictions = stack_model.predict(X_test)

mse = mean_squared_error(y_test, predictions, squared=False)
print(f'RMSE: {np.sqrt(mse)}')

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_stacking.py:957: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.035268 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26014
[LightGBM] [Info] Number of data points in the train set: 45360, number of used features: 104
[LightGBM] [Info] Start training from score 0.685492
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024943 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26014
[LightGBM] [Info] Number of data points in the train set: 36288, number of used features: 104
[LightGBM] [Info] Start training from score 0.685640
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024989 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26014
[LightGBM] [Info] Number of data points in the train set: 36288, number of used features: 104
[LightGBM] [Info] Sta